I'd like to compare the efficiency of vectorization implementation in practice.

0. weight 및 input은 numpy library의 난수 생성 모듈을 활용함: https://numpy.org/doc/2.1/reference/random/generated/numpy.random.rand.html

In [1]:
import numpy as np

In [3]:
weight = [np.random.rand() for _ in range(4)]
weight

[0.2659644009493185,
 0.28592990602353674,
 0.45852195411908314,
 0.12322277485256183]

In [4]:
input  = [np.random.rand() for _ in range(4)]
input

[0.24469178373827627,
 0.8938710781417305,
 0.6767339310464625,
 0.10122556721909737]

`np.random.rand()`

1. totally manual code.

In [10]:
result_1 = weight[0] * input[0] + weight[1] * input[1] + weight[2] * input[2] + weight[3] * input[3]
result_1

0.6434344368102679

2. using for-loop

In [13]:
result_2 = 0

for i in range(len(weight)):
  result_2 += weight[i] * input[i]

result_2

0.6434344368102679

3. using vector

In [14]:
result_3 = np.matmul(weight, input)
result_3

np.float64(0.6434344368102679)

- weighted-sum 결과 비교

In [15]:
result_1 == result_2  # <class 'bool'>

True

In [16]:
result_1 == result_3  # <class 'numpy.bool_'>

np.True_

In [18]:
result_2 == result_3

np.True_

In [19]:
type(result_3)

numpy.float64

In [20]:
type(result_2)

float

- experiment : `n = 100,000,000`

1. totally manual implementation

In [32]:
n = 100000000 # 100 M
# w = [np.random.rand() for _ in range(n)]
# x = [np.random.rand() for _ in range(n)]
w = np.random.rand(n)
x = np.random.rand(n)

len(w)

100000000

In [30]:
result_01 = 0

for i in range(len(w)):
  result_01 += w[i] * x[i]

result_01

np.float64(25003217.113946553)

In [31]:
result_02 = np.matmul(w, x)

result_02

np.float64(25003217.113939464)

- 결과 비교!!


    *   난수 생성에 6초
    *   순차적 연산에 50초
    *   병렬적 연산에 0.n초 남짓 소요됨을 확인 !!

